In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker

# Data cleaning, sorting, and merging of df1, df2, and df3
df1 = pd.read_csv('WLD_RTFP_country_2023-10-02.csv')
df2 = pd.read_csv('world food production.csv')
df3 = pd.read_csv('climate_change_impact_on_agriculture_2024.csv')
df4 = pd.read_csv('population.csv')

# changing date to datetime for sorting
df1['date'] = pd.to_datetime(df1['date'])
df1.sort_values(by=['country', 'date'], inplace=True)

#checking how many rows dont have NaN
rows_with_nan1 = df1[df1.isna().any(axis=1)]
rows_with_nan2 = df2[df2.isna().any(axis=1)]
rows_with_nan3 = df3[df3.isna().any(axis=1)]
rows_with_nan4 = df4[df4.isna().any(axis=1)]

print('The amount of rows with no value in the first dataset is:', len(rows_with_nan1))
print('The amount of rows with no value in the second dataset is:', len(rows_with_nan2))
print('The amount of rows with no value in the third dataset is:', len(rows_with_nan3))
print('The amount of rows with no value in the fourth dataset is:', len(rows_with_nan4))



# cleaning
df1['Inflation'] = df1.groupby('country')['Inflation'].transform(
    lambda group: group.interpolate(method='linear', limit_direction='backward')
)

df1.dropna(inplace = True)


#dropping any duplicate rows in all data sets
df1.drop_duplicates(inplace=True)
df2.drop_duplicates(inplace=True)
df3.drop_duplicates(inplace=True)


# cutting data from the second set to merge onto the 4th

df2 = df2[df2['Year'] > 1990]
###################################

# sorting df3 so that we can compare countries across different crops over time
df3.sort_values(by=['Country', 'Year', 'Crop_Type'], inplace=True)


# sorting data to fit within our time range

df3 = df3[df3['Year'] < 2021]
df2['Total Food'] = df2.iloc[:, 2:].sum(axis=1)

####################################################

# cutting data from 4th to be able to merge onto 2nd

df4 = df4[df4['Year'] > 1990]
df4 = df4.rename(columns={'all years' : 'Population'})
# merging
df24 = pd.merge(df2, df4[['Entity', 'Year', 'Population']], on = ['Entity', 'Year'], how = 'left')
# creating new metrics so that we can compare countries by food produced per person, eliminates bias in countries with already large population
df24['Food per Person'] = df24['Total Food']/df24['Population']

# seperating data by country

China = df24[df24['Entity'] == 'China']
India = df24[df24['Entity'] == 'India']
United_States = df24[df24['Entity'] == 'United States']
Afghanistan = df24[df24['Entity'] == 'Afghanistan']
Pakistan = df24[df24['Entity'] == 'Pakistan']

# plotting the data across 5 different countries
fig, ax = plt.subplots(figsize=(10, 6)) 

ax.plot(China['Year'], China['Food per Person'], label='China', color='blue', linewidth=2)
ax.plot(India['Year'], India['Food per Person'], label='India', color='red', linewidth=2)
ax.plot(United_States['Year'], United_States['Food per Person'], label='USA', color='green', linewidth=2)
ax.plot(Afghanistan['Year'], Afghanistan['Food per Person'], label='Afghanistan', color='purple', linewidth=2)
ax.plot(Pakistan['Year'], Pakistan['Food per Person'], label='Pakistan', color='gold', linewidth=2)

ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Tonnes of Food per Person', fontsize=12)
ax.set_title('Food Produced per Person Over Time', fontsize=14, fontweight='bold')

ax.grid(True, linestyle='--', alpha=0.6)
ax.tick_params(axis='x', rotation=45)
ax.tick_params(axis='both', labelsize=10)

fig.legend(bbox_to_anchor=(1.15, 0.6))

plt.tight_layout()

#upload csv
df = pd.read_csv("WLD_RTFP_country_2023-10-02.csv")
df['Inflation'].fillna(0, inplace=True)

#convert date to datetime
df['date'] = pd.to_datetime(df['date'])

#create filtered dataframe from 2015 - 2023
df_filtered = df[(df['date'].dt.year >= 2015) & (df['date'].dt.year <= 2023)]
latest_per_year = (df_filtered.sort_values('date').groupby(df_filtered['date'].dt.year, as_index=False).last())

#calculate percentage change of prices
latest_per_year['Price_Change'] = latest_per_year['High'].pct_change() * 100
latest_per_year.fillna(0, inplace=True)

#plotting Inflation vs Price Change (2015–2023, Latest per Year)
plt.figure()
plt.plot(latest_per_year['date'], latest_per_year['Inflation'], label='Inflation', color='red', marker='o')
plt.plot(latest_per_year['date'], latest_per_year['Price_Change'], label='Price Change (%)', color='blue', marker='o')
plt.xlabel('Date')
plt.ylabel('Value / Change (%)')
plt.title('Inflation vs Price Change (2015–2023, Latest per Year)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

#read csv
df = pd.read_csv("world food production.csv")
df.columns = df.columns.str.strip().str.replace(r'\s+', ' ', regex=True)

#selecting the countries
selected_entities = ['Afghanistan', 'Argentina', 'Taiwan', 'China', 'Spain',
                     'Japan', 'Canada', 'United States']

#filtering the dataframe per country and year
filtered_df = df[(df['Entity'].isin(selected_entities)) &
                 (df['Year'] >= 2015) & (df['Year'] <= 2023)]

crop_columns = filtered_df.columns.difference(['Entity', 'Year'])
production_sums = filtered_df.groupby('Entity')[crop_columns].sum().sum(axis=1)

colors = plt.cm.tab20(range(len(production_sums)))

#plotting the pie chart
fig, ax = plt.subplots(figsize=(7, 5))
wedges, texts, autotexts = ax.pie(
    production_sums,
    autopct='%1.1f%%',
    startangle=140,
    colors=colors,
    pctdistance=0.8,
    textprops={'fontsize': 9, 'color': 'black'},
    wedgeprops={'edgecolor': 'black', 'linewidth': 0.5}
)

#plotting the legend
ax.legend(wedges, production_sums.index, title="Country", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1), fontsize=9,frameon=False)

plt.setp(autotexts, size=8)
ax.axis('equal')
plt.title('Total Crop Production by Country (2015–2023)', fontsize=12, pad=15)
plt.tight_layout(rect=[0, 0, 0.85, 1])
plt.show()

# Heatmap of Average Temperature vs Crop Yield 2015-2023

df = pd.read_csv("climate_change_impact_on_agriculture_2024.csv")

df_recent = df[(df['Year'] >= 2015) & (df['Year'] <= 2023)]

# Bin temperature and crop yield data into 10 categories
temp_bins = pd.cut(df['Average_Temperature_C'], bins=10)
yield_bins = pd.cut(df['Crop_Yield_MT_per_HA'], bins=10)

# Pivot table (cross-tabulation)
heatmap_data = pd.crosstab(temp_bins, yield_bins)

plt.figure(figsize=(12, 8))
sns.heatmap(heatmap_data, cmap="YlGnBu", annot=True, fmt="d", linewidths=.5)
plt.title("Heatmap of Average Temperature vs Crop Yield 2015-2023")
plt.xlabel("Crop Yield (MT per HA)")
plt.ylabel("Average Temperature (°C)")
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Violin Plot of Crop Yield Variability by Extreme Weather Events (2015-2023)

df_recent = df[(df['Year'] >= 2015) & (df['Year'] <= 2023)].copy()
crops_of_interest = ['Rice', 'Wheat', 'Corn']
df_filtered = df_recent[df_recent['Crop_Type'].isin(crops_of_interest)].copy()

median_extreme_events = df_filtered['Extreme_Weather_Events'].median()
df_filtered['Weather_Intensity'] = np.where(
    df_filtered['Extreme_Weather_Events'] > median_extreme_events, 
    'High Extreme Weather', 
    'Low Extreme Weather'
)

fig, axes = plt.subplots(1, 3, figsize=(18, 8), sharey=True)
fig.suptitle('Crop Yield Variability by Extreme Weather Events (2015-2023)', fontsize=16, y=1.02)

for i, crop in enumerate(crops_of_interest):
    crop_data = df_filtered[df_filtered['Crop_Type'] == crop]
    
    sns.violinplot(
        ax=axes[i],
        x='Weather_Intensity',
        y='Crop_Yield_MT_per_HA',
        data=crop_data,
        inner='quartile',
        cut=0)
    
    axes[i].set_title(crop)
    axes[i].set_xlabel('')
    axes[i].set_ylabel('Crop Yield (MT/HA)' if i == 0 else '')
    
    medians = crop_data.groupby('Weather_Intensity')['Crop_Yield_MT_per_HA'].median()
    for j, intensity in enumerate(['Low Extreme Weather', 'High Extreme Weather']):
        axes[i].text(
            j, medians[intensity] + 0.1, 
            f'Median: {medians[intensity]:.2f}',
            ha='center',
            va='bottom',
            fontsize=10,
            color='black')

plt.tight_layout()
plt.show()

# Average crop production bar graph
df = pd.read_csv('world food production.csv')  

# Filter the dataset
representative_countries = [
    'South Africa',     # Africa
    'Indonesia',        # Asia
    'France',           # Europe
    'United States',    # North America
    'Argentina',        # South America
    'New Zealand'       # Oceania
]


data_avg = df[
    (df['Year'].between(2015, 2023)) & 
    (df['Entity'].isin(representative_countries))
]

# Calculate the average
avg_production = data_avg.groupby('Entity')[
    ['Wheat Production (tonnes)', 'Rice  Production ( tonnes)', 'Maize Production (tonnes)']
].mean().reset_index()


avg_production.columns = ['Country', 'Wheat', 'Rice', 'Maize']
avg_production = avg_production.set_index('Country')
avg_production = avg_production[['Wheat', 'Rice', 'Maize']]  

# Layout
avg_production.plot(kind='bar', figsize=(12, 6), color=['blue', 'red', 'orange'])
plt.title('Average Crop Production (Wheat, Rice, Maize) from 2015 to 2023')
plt.xlabel('Country')
plt.ylabel('Average Production (tonnes)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.legend(title='Crop')
plt.show()

#filtered for selected countries
selected_countries = ['China', 'India', 'United States', 'Afghanistan', 'Pakistan']
df_filtered = df24[df24['Entity'].isin(selected_countries)]

#created pivot table
pivot_df = df_filtered.pivot_table(
    index='Year',
    columns='Entity',
    values=['Population', 'Total Food']
)

#made columns more readable
pivot_df.columns = [f'{value}_{country}' for value, country in pivot_df.columns]
pivot_df.reset_index(inplace=True)

#output to test if cleaner/readable
print("Pivoted DataFrame:\n", pivot_df.head())
print("\nStacked DataFrame:\n", stacked_df.head())